#### Question 1, 2

In [5]:
def readGenome(filename):
    '''
    Reads the reference genome
    '''
    genome=''
    with open(filename,'r') as f:
        for line in f:
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

genome = readGenome('chr1.GRCh38.excerpt.fasta')

In [9]:
genome[:30]

'TTGAATGCTGAAATCAGCAGGTAATATATG'

In [62]:
def naive(read, genome):
    match = True
    alignment = 0
    comparison = 0
    match_list = []
    for i in range(len(genome) - len(read) + 1):
        alignment += 1
        for j in range(len(read)):
            comparison += 1
            if not read[j] == genome[i + j]:
                match = False
                break
        if match:
            match_list.append(i)
    return match_list, alignment, comparison

In [63]:
match_list, alignment, comparison = naive('GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG', genome)

In [64]:
print(match_list, alignment, comparison)

[] 799954 984143


In [18]:
len(genome)

800000

#### Question 3

In [1]:
import string

def z_array(s):
    """ Use Z algorithm (Gusfield theorem 1.4.1) to preprocess s """
    assert len(s) > 1
    z = [len(s)] + [0] * (len(s)-1)
    # Initial comparison of s[1:] with prefix
    for i in range(1, len(s)):
        if s[i] == s[i-1]:
            z[1] += 1
        else:
            break
    r, l = 0, 0
    if z[1] > 0:
        r, l = z[1], 1
    for k in range(2, len(s)):
        assert z[k] == 0
        if k > r:
            # Case 1
            for i in range(k, len(s)):
                if s[i] == s[i-k]:
                    z[k] += 1
                else:
                    break
            r, l = k + z[k] - 1, k
        else:
            # Case 2
            # Calculate length of beta
            nbeta = r - k + 1
            zkp = z[k - l]
            if nbeta > zkp:
                # Case 2a: Zkp wins
                z[k] = zkp
            else:
                # Case 2b: Compare characters just past r
                nmatch = 0
                for i in range(r+1, len(s)):
                    if s[i] == s[i - k]:
                        nmatch += 1
                    else:
                        break
                l, r = k, r + nmatch
                z[k] = r - k + 1
    return z


def n_array(s):
    """ Compile the N array (Gusfield theorem 2.2.2) from the Z array """
    return z_array(s[::-1])[::-1]


def big_l_prime_array(p, n):
    """ Compile L' array (Gusfield theorem 2.2.2) using p and N array.
        L'[i] = largest index j less than n such that N[j] = |P[i:]| """
    lp = [0] * len(p)
    for j in range(len(p)-1):
        i = len(p) - n[j]
        if i < len(p):
            lp[i] = j + 1
    return lp


def big_l_array(p, lp):
    """ Compile L array (Gusfield theorem 2.2.2) using p and L' array.
        L[i] = largest index j less than n such that N[j] >= |P[i:]| """
    l = [0] * len(p)
    l[1] = lp[1]
    for i in range(2, len(p)):
        l[i] = max(l[i-1], lp[i])
    return l


def small_l_prime_array(n):
    """ Compile lp' array (Gusfield theorem 2.2.4) using N array. """
    small_lp = [0] * len(n)
    for i in range(len(n)):
        if n[i] == i+1:  # prefix matching a suffix
            small_lp[len(n)-i-1] = i+1
    for i in range(len(n)-2, -1, -1):  # "smear" them out to the left
        if small_lp[i] == 0:
            small_lp[i] = small_lp[i+1]
    return small_lp


def good_suffix_table(p):
    """ Return tables needed to apply good suffix rule. """
    n = n_array(p)
    lp = big_l_prime_array(p, n)
    return lp, big_l_array(p, lp), small_l_prime_array(n)


def good_suffix_mismatch(i, big_l_prime, small_l_prime):
    """ Given a mismatch at offset i, and given L/L' and l' arrays,
        return amount to shift as determined by good suffix rule. """
    length = len(big_l_prime)
    assert i < length
    if i == length - 1:
        return 0
    i += 1  # i points to leftmost matching position of P
    if big_l_prime[i] > 0:
        return length - big_l_prime[i]
    return length - small_l_prime[i]


def good_suffix_match(small_l_prime):
    """ Given a full match of P to T, return amount to shift as
        determined by good suffix rule. """
    return len(small_l_prime) - small_l_prime[1]


def dense_bad_char_tab(p, amap):
    """ Given pattern string and list with ordered alphabet characters, create
        and return a dense bad character table.  Table is indexed by offset
        then by character. """
    tab = []
    nxt = [0] * len(amap)
    for i in range(0, len(p)):
        c = p[i]
        assert c in amap
        tab.append(nxt[:])
        nxt[amap[c]] = i+1
    return tab


class BoyerMoore(object):
    """ Encapsulates pattern and associated Boyer-Moore preprocessing. """
    
    def __init__(self, p, alphabet='ACGT'):
        self.p = p
        self.alphabet = alphabet
        # Create map from alphabet characters to integers
        self.amap = {}
        for i in range(len(self.alphabet)):
            self.amap[self.alphabet[i]] = i
        # Make bad character rule table
        self.bad_char = dense_bad_char_tab(p, self.amap)
        # Create good suffix rule table
        _, self.big_l, self.small_l_prime = good_suffix_table(p)
    
    def bad_character_rule(self, i, c):
        """ Return # skips given by bad character rule at offset i """
        assert c in self.amap
        ci = self.amap[c]
        assert i > (self.bad_char[i][ci]-1)
        return i - (self.bad_char[i][ci]-1)
    
    def good_suffix_rule(self, i):
        """ Given a mismatch at offset i, return amount to shift
            as determined by (weak) good suffix rule. """
        length = len(self.big_l)
        assert i < length
        if i == length - 1:
            return 0
        i += 1  # i points to leftmost matching position of P
        if self.big_l[i] > 0:
            return length - self.big_l[i]
        return length - self.small_l_prime[i]
    
    def match_skip(self):
        """ Return amount to shift in case where P matches T """
        return len(self.small_l_prime) - self.small_l_prime[1]

In [68]:
def boyer_moore(p, p_bm, t):
    occurence = []
    alignment_total = 0
    i = 0
    while i < len(t) - len(p) + 1:
        alignment_total += 1
        shift = 1
        match = True
        for j in range(len(p) - 1, -1, -1):
            if not t[i + j] == p[j]:
                gs = p_bm.good_suffix_rule(j)
                bd = p_bm.bad_character_rule(j, t[i+j])
                match = False
                shift = max(gs, bd, shift)
                break
        if match:
            ms = p_bm.match_skip()
            shift = max(ms, shift)
            occurence.append(i)
        i += shift
    return occurence, alignment_total

t = 'GCTAGA'
p = 'TCTA'
p_bm = BoyerMoore(p, alphabet='ACGT')
occurence, alignments = boyer_moore(p, p_bm, t)

In [70]:
def boyer_moore(p, p_bm, t):
    """ Do Boyer-Moore matching """
    i = 0
    occurrences = []
    # (len(t) - len(p) + 1) is the maximum number of possible alignments
    while i < len(t) - len(p) + 1:
        shift = 1
        mismatched = False
        # try character comparison BACKWARDS
        for j in range(len(p)-1, -1, -1):
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                # once mismatch occurs, record the number of shifts before break
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return occurrences

read = 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'
p_bm = BoyerMoore(read, alphabet='ACGT')
occurence = boyer_moore(read, p_bm, genome)
print(occurence)

[56922]


In [ ]:
'GCTAGA'
'TCTA'
  'TCTA'

In [66]:
print(occurence,alignments)

[] 1


In [69]:
read = 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'
p_bm = BoyerMoore(read, alphabet='ACGT')
occurence, alignments = boyer_moore(read, p_bm, genome)
print(occurence,alignments)

[56922] 127974


#### Question 4

In [21]:
import bisect

class Index(object):
    """ Holds a substring index for a text T """
    def __init__(self, t, k):
        """ Create index from all substrings of t of length k """
        self.t = t
        self.k = k  # k-mer length (k)
        self.index = []
        for i in range(len(t) - k + 1):  # for each k-mer
            self.index.append((t[i:i+k], i))  # add (k-mer, offset) pair
        self.index.sort()  # alphabetize by k-mer
    def query(self, p):
        """ Return index hits for first k-mer of p """
        kmer = p[:self.k]  # query with first k-mer
        i = bisect.bisect_left(self.index, (kmer, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != kmer:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits

In [23]:
def queryIndex(p, t, index_obj):
    k = index_obj.k
    hits_list = index_obj.query(p)
    offset = []
    for i in hits_list:
        if p[k:] == t[i + k: i + len(p)]:
            offset.append(i)
    return offset

text = 'wegdsatesdsafdsate'
pattern = 'ate'
index_obj = Index(text, 2)
queryIndex(pattern, text, index_obj)

[5, 15]

In [32]:
test= 'sdfsa'
test[1+1:2+2]

'fs'

In [40]:
import bisect

class pigeonhole_kmer(object):
    """ Holds a substring index for a text T """
    def __init__(self, t, k, p, m):
        """ Create index from all substrings of t of length k """
        self.t = t
        self.k = k  # k-mer length (k)
        self.p = p
        self.m = m
        self.index = []
        for i in range(len(t) - k + 1):  # for each k-mer
            self.index.append((t[i:i+k], i))  # add (k-mer, offset) pair
        self.index.sort()  # alphabetize by k-mer
        
    def kmerQuery(self, p_sec):
        """ Return index hits for first k-mer of p """
        kmer = p_sec[:self.k]  # query with first k-mer
        i = bisect.bisect_left(self.index, (kmer, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != kmer:
                break
            hits.append(self.index[i][1])
            i += 1
            
        offset = []
        for i in hits:
            if p_sec[self.k:] == t[i + self.k: i + len(p_sec)]:
                offset.append(i)
        return offset

    def pigeonhole(self):
        p_section_n = self.m + 1
        p_section_l = round(len(self.p) / p_section_n)
        occurence = []
        for i in range(p_section_n):
            start = i * p_section_l
            end = min((i + 1) * p_section_l, len(self.p))
            p_section = self.p[start:end]
            matches = self.kmerQuery(p_section)
            
            for m in matches:
                mismatches = 0
                if m < start or m - start + len(p_section) > len(self.t):
                    continue
                for i in range(0, start):
                    if self.p[i] != self.t[m - start + i]:
                        mismatches += 1
                    if mismatches > self.m:
                        break
                for j in range(end, len(self.p)):
                    if self.p[j] != self.t[m - start + j]:
                        mismatches += 1
                    if mismatches > self.m:
                        break                        
                
                if mismatches <= self.m:
                    occurence.append((m - start, mismatches))
                
        return occurence
    
read = 'GGCGCGGTGGCTCACGCCTGTAAT'                        
pk = pigeonhole_kmer(genome, 8, read, 2)
ls = pk.pigeonhole()
ls = list(set(ls))
ls.sort(key=lambda tup: tup[0])
ls

[(56922, 0),
 (84641, 1),
 (147558, 1),
 (160162, 2),
 (160729, 1),
 (191452, 1),
 (262042, 0),
 (273669, 1),
 (364263, 0),
 (421221, 2),
 (429299, 1),
 (465647, 1),
 (551134, 2),
 (635931, 2),
 (657496, 0),
 (681737, 1),
 (717706, 0),
 (724927, 1),
 (747359, 2)]

In [41]:
len(ls)

19

In [20]:
class naive_ap(object):
    def __init__(self, p, t, m):
        self.p = p
        self.t = t
        self.m = m
        
    def naive(self):
        occurence = []
        for i in range(len(self.t) - len(self.p) + 1):
            match = True
            mismatch = 0
            for j in range(len(self.p)):
                if self.t[i + j ] != self.p[j]:
                    mismatch += 1
                if mismatch > self.m:
                    match = False
                    break
            if match:
                occurence.append((i, mismatch))
        return occurence

read = 'GGCGCGGTGGCTCACGCCTGTAAT'
result = naive_ap(read, genome, 2)
result.naive()

[(56922, 0),
 (84641, 1),
 (147558, 1),
 (160162, 2),
 (160729, 1),
 (191452, 1),
 (262042, 0),
 (273669, 1),
 (364263, 0),
 (421221, 2),
 (429299, 1),
 (465647, 1),
 (551134, 2),
 (635931, 2),
 (657496, 0),
 (681737, 1),
 (717706, 0),
 (724927, 1),
 (747359, 2)]

#### Question5

In [53]:
import bisect

class hits_pigeonhole_kmer(object):
    """ Holds a substring index for a text T """
    def __init__(self, t, k, p, m):
        """ Create index from all substrings of t of length k """
        self.t = t
        self.k = k  # k-mer length (k)
        self.p = p
        self.m = m
        self.index = []
        for i in range(len(t) - k + 1):  # for each k-mer
            self.index.append((t[i:i+k], i))  # add (k-mer, offset) pair
        self.index.sort()  # alphabetize by k-mer
        
    def kmerQuery(self, p_sec):
        """ Return index hits for first k-mer of p """
        kmer = p_sec[:self.k]  # query with first k-mer
        i = bisect.bisect_left(self.index, (kmer, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != kmer:
                break
            hits.append(self.index[i][1])
            i += 1
            
        offset = []
        for i in hits:
            if p_sec[self.k:] == t[i + self.k: i + len(p_sec)]:
                offset.append(i)
        return hits, offset

    def pigeonhole(self):
        p_section_n = self.m + 1
        p_section_l = round(len(self.p) / p_section_n)
        total_hits = []
        for i in range(p_section_n):
            start = i * p_section_l
            end = min((i + 1) * p_section_l, len(self.p))
            p_section = self.p[start:end]
            hits, _ = self.kmerQuery(p_section)
            total_hits.extend(hits)
        return len(total_hits)

read = 'GGCGCGGTGGCTCACGCCTGTAAT'                        
ht = hits_pigeonhole_kmer(genome, 8, read, 2)
ht.pigeonhole()

90

In [58]:
test = 'sfasfdgfsdadfs'
test[1:5:1]

'fasf'

In [60]:
import bisect
   
class SubseqIndex(object):
    """ Holds a subsequence index for a text T """
    
    def __init__(self, t, k, ival):
        """ Create index from all subsequences consisting of k characters
            spaced ival positions apart.  E.g., SubseqIndex("ATAT", 2, 2)
            extracts ("AA", 0) and ("TT", 1). """
        self.k = k  # num characters per subsequence extracted
        self.ival = ival  # space between them; 1=adjacent, 2=every other, etc
        self.index = []
        self.span = 1 + ival * (k - 1)
        for i in range(len(t) - self.span + 1):  # for each subseq
            self.index.append((t[i:i+self.span:ival], i))  # add (subseq, offset)
        self.index.sort()  # alphabetize by subseq
    
    def query(self, p):
        """ Return index hits for first subseq of p """
        subseq = p[:self.span:self.ival]  # query with first subseq
        i = bisect.bisect_left(self.index, (subseq, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != subseq:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits

read = 'GGCGCGGTGGCTCACGCCTGTAAT' 
si = SubseqIndex(genome, 8, 3)
len(si.query(read))

35